# Работа со строковыми значениями

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Работа со строковыми значениям"
* https://pyformat.info/
* https://docs.python.org/3/library/re.html
    * https://docs.python.org/3/library/re.html#flags
    * https://docs.python.org/3/library/re.html#functions
* https://pythonru.com/primery/primery-primeneniya-regulyarnyh-vyrazheniy-v-python
* https://kanoki.org/2019/11/12/how-to-use-regex-in-pandas/
* https://realpython.com/nltk-nlp-python/

## Задачи для совместного разбора

1. Вывести на экран данные из словаря `obj` построчно в виде `k = v`, задав формат таким образом, чтобы знак равенства оказался на одной и той же позиции во всех строках. Строковые литералы обернуть в кавычки.

In [1]:
obj = {
    "home_page": "https://github.com/pypa/sampleproject",
    "keywords": "sample setuptools development",
    "license": "MIT",
}

2. Написать регулярное выражение,которое позволит найти номера групп студентов.

In [4]:
obj = pd.Series(["Евгения гр.ПМ19-1", "Илья пм 20-4", "Анна 20-3"])
obj

0    Евгения гр.ПМ19-1
1         Илья пм 20-4
2            Анна 20-3
dtype: object

3. Разбейте текст формулировки задачи 2 на слова.

## Лабораторная работа 6

### Форматирование строк

1\. Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в виде `pd.DataFrame` `recipes` При помощи форматирования строк выведите информацию об id рецепта и времени выполнения 5 случайных рецептов в виде таблицы следующего вида:

    
    |      id      |  minutes  |
    |--------------------------|
    |    61178     |    65     |
    |    202352    |    80     |
    |    364322    |    150    |
    |    26177     |    20     |
    |    224785    |    35     |
    
Обратите внимание, что ширина столбцов заранее неизвестна и должна рассчитываться динамически, в зависимости от тех данных, которые были выбраны. 

In [1]:
import pandas as pd
import numpy as np

# Загрузка данных
recipes = pd.read_csv('recipes_sample.csv')

# Выбор 5 случайных рецептов
sample_recipes = recipes.sample(5)

# Определение максимальной ширины столбцов
max_id_width = max(sample_recipes['id'].astype(str).map(len).max(), len('id'))
max_minutes_width = max(sample_recipes['minutes'].astype(str).map(len).max(), len('minutes'))

# Форматирование и вывод таблицы
print(f"|{'id'.center(max_id_width)}|{'minutes'.center(max_minutes_width)}|")
print('-' * (max_id_width + max_minutes_width + 3))

for index, row in sample_recipes.iterrows():
    id_str = str(row['id']).center(max_id_width)
    minutes_str = str(row['minutes']).center(max_minutes_width)
    print(f"|{id_str}|{minutes_str}|")


|  id  |minutes|
----------------
|269516|   45  |
|115721|   25  |
|258749|   45  |
|234023|   12  |
|176112|   5   |


2\. Напишите функцию `show_info`, которая по данным о рецепте создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название Из Нескольких Слов"

1. Шаг 1
2. Шаг 2
----------
Автор: contributor_id
Среднее время приготовления: minutes минут
```

    
Данные для создания строки получите из файлов `recipes_sample.csv` (__ЛР2__) и `steps_sample.xml` (__ЛР3__). 
Вызовите данную функцию для рецепта с id `170895` и выведите (через `print`) полученную строку на экран.

In [13]:
assert (
    show_info(
        name="george s at the cove black bean soup",
        steps=[
            "clean the leeks and discard the dark green portions",
            "cut the leeks lengthwise then into one-inch pieces",
            "melt the butter in a medium skillet , med",
        ],
        minutes=90,
        author_id=35193,
    )
    == '"George S At The Cove Black Bean Soup"\n\n1. Clean the leeks and discard the dark green portions\n2. Cut the leeks lengthwise then into one-inch pieces\n3. Melt the butter in a medium skillet , med\n----------\nАвтор: 35193\nСреднее время приготовления: 90 минут\n'
)

<IPython.core.display.Javascript object>

In [2]:
def show_info(name, steps, minutes, author_id):
    # Форматирование названия рецепта с заглавной буквы каждого слова
    title = name.title()

    # Создание отформатированного списка шагов
    steps_formatted = '\n'.join(f"{i+1}. {step.capitalize()}" for i, step in enumerate(steps))

    # Сборка итоговой строки
    info = f'"{title}"\n\n{steps_formatted}\n----------\nАвтор: {author_id}\nСреднее время приготовления: {minutes} минут\n'
    return info

# Пример использования функции
recipe_info = show_info(
    name="george s at the cove black bean soup",
    steps=[
        "clean the leeks and discard the dark green portions",
        "cut the leeks lengthwise then into one-inch pieces",
        "melt the butter in a medium skillet , med",
    ],
    minutes=90,
    author_id=35193,
)

print(recipe_info)


"George S At The Cove Black Bean Soup"

1. Clean the leeks and discard the dark green portions
2. Cut the leeks lengthwise then into one-inch pieces
3. Melt the butter in a medium skillet , med
----------
Автор: 35193
Среднее время приготовления: 90 минут



## Работа с регулярными выражениями

3\. Напишите регулярное выражение, которое ищет следующий паттерн в строке: число (1 цифра или более), затем пробел, затем слова: hour или hours или minute или minutes. Произведите поиск по данному регулярному выражению в каждом шаге рецепта с id 25082. Выведите на экран все непустые результаты, найденные по данному шаблону.

In [9]:
import pandas as pd

# Загрузите данные в DataFrame
df = pd.read_csv('recipes_sample.csv')

# Регулярное выражение для поиска шаблона "this..., but" в начале строки
pattern = r'^this[\w\s]*, ?but'

# Используйте строковые методы pd.Series для поиска шаблона с параметром na=False
matches = df['description'].str.contains(pattern, regex=True, na=False)

# Подсчитайте количество рецептов с данным шаблоном
count = matches.sum()
print(f'Количество рецептов с шаблоном "this..., but": {count}')

# Выведите на экран 3 примера описаний
examples = df.loc[matches, 'description'].head(3)
for i, example in enumerate(examples, 1):
    print(f'Пример {i}: {example}')


Количество рецептов с шаблоном "this..., but": 134
Пример 1: this is a great meal eaten the same day ,but even better the next day , if you can wait! add your favourite spices, but try it first as it is and i think that you will enjoy the 'vegetable' taste. good for freezing.
Пример 2: this was adapted from a recipe i found on the net, but i added julienne onion to the peppers.  this is a meal in itself, or you could have a small slice with a meat dish.  for those that like to have brunch, it's a bit different to your traditional quiche recipes.  if you love cheese, you could add 1/2 cup of your favorite to the egg mixture, then pour over peppers.
Пример 3: this is kind of similar to some of the other versions out there, but it is the best and easiest i have found


4\. Напишите регулярное выражение, которое ищет шаблон вида "this..., but" _в начале строки_ . Между словом "this" и частью ", but" может находиться произвольное число букв, цифр, знаков подчеркивания и пробелов. Никаких других символов вместо многоточия быть не может. Пробел между запятой и словом "but" может присутствовать или отсутствовать.

Используя строковые методы `pd.Series`, выясните, для каких рецептов данный шаблон содержится в тексте описания. Выведите на экран количество таких рецептов и 3 примера подходящих описаний (текст описания должен быть виден на экране полностью).

In [10]:
import pandas as pd

# Загрузите данные в DataFrame
df = pd.read_csv('recipes_sample.csv')

# Регулярное выражение для поиска шаблона "this..., but" в начале строки
pattern = r'^this[\w\s]*, ?but'

# Используйте строковые методы pd.Series для поиска шаблона с параметром na=False
matches = df['description'].str.contains(pattern, regex=True, na=False)

# Подсчитайте количество рецептов с данным шаблоном
count = matches.sum()
print(f'Количество рецептов с шаблоном "this..., but": {count}')

# Выведите на экран 3 примера описаний
examples = df.loc[matches, 'description'].head(3)
for i, example in enumerate(examples, 1):
    print(f'Пример {i}: {example}')


Количество рецептов с шаблоном "this..., but": 134
Пример 1: this is a great meal eaten the same day ,but even better the next day , if you can wait! add your favourite spices, but try it first as it is and i think that you will enjoy the 'vegetable' taste. good for freezing.
Пример 2: this was adapted from a recipe i found on the net, but i added julienne onion to the peppers.  this is a meal in itself, or you could have a small slice with a meat dish.  for those that like to have brunch, it's a bit different to your traditional quiche recipes.  if you love cheese, you could add 1/2 cup of your favorite to the egg mixture, then pour over peppers.
Пример 3: this is kind of similar to some of the other versions out there, but it is the best and easiest i have found


5\. В текстах шагов рецептов обыкновенные дроби имеют вид "a / b". Используя регулярные выражения, уберите в тексте шагов рецепта с id 72367 пробелы до и после символа дроби. Выведите на экран шаги этого рецепта после их изменения.

In [14]:
import xml.etree.ElementTree as ET
import re

# Загрузите XML-файл
tree = ET.parse('steps_sample.xml')

# Найдите рецепт с id 72367
recipe = tree.find('.//step[@id="72367"]')

# Проверьте, существует ли рецепт с указанным id
if recipe is None:
    print('Рецепт с указанным id не найден.')
else:
    # Извлеките текст шагов
    steps = recipe.text

    # Замените пробелы до и после символа дроби с помощью регулярного выражения
    steps = re.sub(r' (?=/)|(?<=/ )', '', steps)

    # Обновите текст шагов в XML-файле
    recipe.text = steps

    # Выведите на экран измененные шаги
    print(steps)



Рецепт с указанным id не найден.


### Сегментация текста

6\. Разбейте тексты шагов рецептов на слова при помощи пакета `nltk`. Посчитайте и выведите на экран кол-во уникальных слов среди всех рецептов. Словом называется любая последовательность алфавитных символов (для проверки можно воспользоваться `str.isalpha`). При подсчете количества уникальных слов не учитывайте регистр.

In [18]:
python -m nltk.downloader punkt

SyntaxError: invalid syntax (976932090.py, line 1)

In [19]:
import nltk
from nltk.tokenize import word_tokenize
from collections import Counter

# Загрузите данные рецептов
recipes = pd.read_csv('recipes_sample.csv')

# Инициализируйте список для хранения всех слов из всех рецептов
all_words = []

# Для каждого рецепта
for recipe in recipes['description']:
    # Токенизируйте текст рецепта на слова
    words = word_tokenize(recipe.lower())
    
    # Отфильтруйте слова, содержащие только алфавитные символы
    words = [word for word in words if word.isalpha()]
    
    # Добавьте слова в список всех слов
    all_words.extend(words)

# Подсчитайте количество уникальных слов
unique_words = len(set(all_words))

# Выведите на экран количество уникальных слов
print(f'Количество уникальных слов: {unique_words}')



LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/english.pickle[0m

  Searched in:
    - 'C:\\Users\\lygwu/nltk_data'
    - 'C:\\Users\\lygwu\\anaconda3\\nltk_data'
    - 'C:\\Users\\lygwu\\anaconda3\\share\\nltk_data'
    - 'C:\\Users\\lygwu\\anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\lygwu\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - ''
**********************************************************************


7\. Разбейте описания рецептов из `recipes` на предложения при помощи пакета `nltk`. Найдите 5 самых длинных описаний (по количеству _предложений_) рецептов в датасете и выведите строки фрейма, соответствующие этим рецептами, в порядке убывания длины.

In [20]:
import nltk
from nltk.tokenize import sent_tokenize

# Загрузите данные рецептов
recipes = pd.read_csv('recipes_sample.csv')

# Инициализируйте список для хранения количества предложений в каждом рецепте
num_sentences = []

# Для каждого рецепта
for recipe in recipes['description']:
    # Токенизируйте описание рецепта на предложения
    sentences = sent_tokenize(recipe)
    
    # Добавьте количество предложений в список
    num_sentences.append(len(sentences))

# Создайте новый столбец с количеством предложений в фрейме данных
recipes['num_sentences'] = num_sentences

# Отсортируйте фрейм данных по количеству предложений в убывающем порядке
recipes = recipes.sort_values('num_sentences', ascending=False)

# Выведите 5 самых длинных описаний рецептов
print(recipes.head(5))


LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/english.pickle[0m

  Searched in:
    - 'C:\\Users\\lygwu/nltk_data'
    - 'C:\\Users\\lygwu\\anaconda3\\nltk_data'
    - 'C:\\Users\\lygwu\\anaconda3\\share\\nltk_data'
    - 'C:\\Users\\lygwu\\anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\lygwu\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - ''
**********************************************************************


8\. Напишите функцию, которая для заданного предложения выводит информацию о частях речи слов, входящих в предложение, в следующем виде:
```
PRP   VBD   DT      NNS     CC   VBD      NNS        RB   
 I  omitted the raspberries and added strawberries instead
``` 
Для определения части речи слова можно воспользоваться `nltk.pos_tag`.

Проверьте работоспособность функции на названии рецепта с id 241106.

Обратите внимание, что часть речи должна находиться ровно посередине над соотвествующим словом, а между самими словами должен быть ровно один пробел.


In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag

def pos_tag_sentence(sentence):
    tokens = word_tokenize(sentence)
    pos_tags = pos_tag(tokens)
    return pos_tags

try:
    pos_tag_sentence('')
except LookupError:
    import nltk
    nltk.download('averaged_perceptron_tagger')

def process(pos_tags):
    output = ""
    for token, pos_tag in pos_tags:
        output += f"{pos_tag:<5}"
    output += "\n"

    for token, _ in pos_tags:
        output += f"{token} "

    return output


recipe_id = 241106
recipes = pd.read_csv('recipes_sample.csv', header=0)
recipe_data = recipes[recipes['id'] == recipe_id]

print(process(pos_tag_sentence(recipe_data['name'].values[0])))
